In [8]:
# connect to google drive and move to the selected directory
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/Stage LIP6/CBOW"
!pwd 
!ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Stage LIP6/CBOW
/content/gdrive/My Drive/Stage LIP6/CBOW
data	     models.py	__pycache__  runs	 test.py
datasets.py  old	README.md    test.ipynb


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import trange
import matplotlib.pyplot as plt
import sys

import models
from datasets import CBOWDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

writer=SummaryWriter("runs/tensorboard")

def main():
    torch.manual_seed(1)
    CUDA = torch.cuda.is_available()
    if CUDA:
        print("avaible GPUs:", torch.cuda.device_count())
        print("GPU name:", torch.cuda.get_device_name())
    print("pytorch version: ", torch.__version__)

    dataset = CBOWDataset("data/en.txt",context_size=2,max_len=10)

    # Parameters
    CONTEXT_SIZE = 2
    EMBEDDING_SIZE = 300
    EPOCHS = 5
    LEARNING_RATE = 0.001

    # Model
    model = models.CBOW(len(dataset)+2*CONTEXT_SIZE, EMBEDDING_SIZE, CONTEXT_SIZE)
    if CUDA:
        model = model.cuda()

    #### testing tensor board
    # data_loader=DataLoader(dataset,batch_size=2,shuffle=True)
    # examples=iter(data_loader)
    # example_context,example_target=examples.next()
    example_context,example_target=dataset[0]

    writer.add_graph(model, example_context)
    writer.close()
    sys.exit()
    # Training
    losses = train(model, dataset, EPOCHS, LEARNING_RATE)
    print(losses)


def train(model, dataset, epochs, lr):
    '''
    Train a model 

    Parameters:
        model (nn.module):
        data(list of tuples):  
        words_to_idx() :dict containing a mapping word->index
    Return:
        tensor (Tensor): 
    '''
    loss_func = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    losses = []

    for epoch in trange(epochs):
        total_loss = 0
        for i in range(len(dataset)):
            context=dataset[i][0]
            target=dataset[i][1]
            output = model(context)
            loss = loss_func(output, target)

            model.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        losses.append(total_loss)
    return losses

if __name__ == '__main__':
    main()

avaible GPUs: 1
GPU name: Tesla T4
pytorch version:  1.6.0+cu101
file:  <_io.TextIOWrapper name='data/en.txt' mode='r' encoding='utf8'>


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
!python test.py

python3: can't open file 'test.py': [Errno 2] No such file or directory


In [ ]:
!tensorboard --logdir=runs

2020-08-25 05:14:14.674248: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.3.0 at http://localhost:6006/ (Press CTRL+C to quit)
